# Введение

## Описание задачи
[источник](https://contest.yandex.ru/contest/20144/problems/)

### RUS

Каждый пользователь, размещая объявление об аренде квартиры, хочет понимать, сколько времени потребуется для сдачи объекта. В первую очередь такая информация нужна на форме подачи объявлений, что заметно ограничивает набор возможных признаков. Вам предлагается построить модель, прогнозирующую длительность экспозиции объявлений на Яндекс.Недвижимости.
Для построения целевой переменной срок экспозиции разбит на несколько классов, каждому из которых соответствует целое число: "меньше 7 дней"(1), "7-14 дней"(2), "15-30 дней"(3), "30-70 дней"(4), "более 70 дней"(5).
Метрика, по которой оцениваются решения, записывается следующим образом:
$$
metric = -\frac{1}{l}\sum\limits_{i=1}^l \left(exp^{|prediction_i - target_i|} - 1\right)
$$

При отправке решения метрика вычисляется на публичной части тестовой выборки. Финальное значение будет определено по скрытой части тестовой выборки, чтобы исключить переобучение. Это значение будет опубликовано после окончания хакатона.
Оценивается только последняя посылка. Убедитесь, что на момент окончания хакатона ваша последняя посылка содержит именно тот прогноз, для которого вы хотите, чтобы было посчитано финальное значение метрики на скрытой части тестовой выборки.

## Описание датасета

### RUS

* **id** - id объявления про аренду квартиры
* **building_id** - id дома
* **unified_address** - адрес дома
* **building_series_id** - id серии дома
* **build_year** - год постройки дома
* **site_id** - id жилого комплекса, если у недавно построенных домов
* **parking** - тип парковки дома
* **expect_demolition** - дом входит в программу реновации и ожидает сноса
* **flats_count** - количество квартир в доме	
* **building_type** - тип стен в доме
* **main_image** - часть урла главной фотографии в объявлении, добавив "http:" можно получить урл (http://avatars.mds.yandex.net/get-realty/1702013/add.d352c435e10b2c47092f43a332bedb13.realty-api-vos/main/)
* **latitude, longitude** - координаты дома
* **total_area** - площадь квартиры в кв. м.
* **ceiling_height** - высота потолков в квартире
* **rooms** - количество комнат в квартире
* **floors_total** - количество этажей в доме
* **floor** - этаж квартиры
* **living_area** - жилая площадь в квартире
* **kitchen_area** - площадь кухни
* **is_apartment** - квартира юридически оформлена как апартаменты
* **studio** - квартиры является студией
* **has_elevator** - наличие лифта в доме	
* **day** - первый день экспозиции квартиры
* **balcony** - тип и наличие балкона
* **renovation** - качество ремонта
* **locality_name** - имя населенного пункта
* **price** - цена аренды квартиры за 1 месяц
* **target** - срок экспозиции, который надо научиться предсказывать
* **target_string** - строковое представление срока экспозиции

# Подготовка ноутбука

## Первичные константы 

In [1]:
PROJECT_NAME = "hack_the_realty_exposition"
MOUNT_DIR = '/content/drive' # In case Colab Usage
VALIDATE_RATIO = 0.2

## Дополнительные установки

In [2]:
!pip install catboost

## Библиотеки

In [3]:
import os

from tqdm import tqdm

from datetime import datetime

from collections import Counter

import geopy.distance
from geopy.geocoders import Nominatim

import math

import numpy as np

import pandas as pd

import catboost as ctb
from catboost import CatBoostRegressor, Pool
from catboost.utils import get_roc_curve

from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import roc_auc_score

from plotly.subplots import make_subplots
import plotly.graph_objects as go

import matplotlib.pyplot as plt

%matplotlib inline

## Обработка случая работы в Google.Colab

### Подключение библиотек

In [4]:
try:
    from google.colab import files, drive
    
    USE_COLAB = True
except:
    USE_COLAB = False

if USE_COLAB:
    print("Don't forget to avoid disconnections:")
    print("""
function ClickConnect(){
    console.log("Clicking"); 
    document.querySelector("colab-connect-button").click() 
}
setInterval(ClickConnect,60000)
    """)

Don't forget to avoid disconnections:

function ClickConnect(){
    console.log("Clicking"); 
    document.querySelector("colab-connect-button").click() 
}
setInterval(ClickConnect,60000)
    


### Подключение к Google.Drive

In [5]:
if USE_COLAB:
    drive.mount(MOUNT_DIR)
    DRIVE_DIR = os.path.join(MOUNT_DIR, 'My Drive')
    print(f"Drive directory is {DRIVE_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive directory is /content/drive/My Drive


## Объявление рабочей директории

Подключение к Google.Drive в случае работы c Google.Colab

In [6]:
PROJECT_DIR = os.path.join(DRIVE_DIR, 'projects', PROJECT_NAME) if USE_COLAB else './'
WORK_DIR = '/content' if USE_COLAB else PROJECT_DIR
print(f"Project directory is {PROJECT_DIR}")
print(f"Working directory is {WORK_DIR}")

Project directory is /content/drive/My Drive/projects/hack_the_realty_exposition
Working directory is /content


# Обработка данных

На выходе должны быть объявлены переменные:

* **X_train_pure**: pd.DataFrame\
id | building_series_id | site_id | parking | build_year | expect_demolition | ~~main_image~~ | latitude | ~~total_area~~ | ceiling_height | rooms | floors_total | living_area | floor | is_apartment | ~~building_id~~ | has_elevator | studio | unified_address | area | kitchen_area | ~~day~~ | longitude | price | flats_count | building_type | balcony | locality_name | renovation | *Month* | *Day* | *DayOfWeek* | *Season* | *District* | *PricePerArea* | *FloorPercantage* | *FloorHeight* | *MainImageNa*

* **y_train**: pd.Series\
id | target
* **X_valid_pure**: pd.DataFrame\
id | building_series_id | site_id | parking | build_year | expect_demolition | latitude | total_area | ceiling_height | rooms | floors_total | living_area | floor | is_apartment | building_id | has_elevator | studio | unified_address | area | kitchen_area | day | longitude | price | flats_count | building_type | balcony | locality_name | renovation 
* **y_valid**: pd.Series\
id | target
* **X_test_pure**: pd.DataFrame\
id | building_series_id | site_id | parking | build_year | expect_demolition | latitude | total_area | ceiling_height | rooms | floors_total | living_area | floor | is_apartment | building_id | has_elevator | studio | unified_address | area | kitchen_area | day | longitude | price | flats_count | building_type | balcony | locality_name | renovation 

## Загрузка

Загрузка train/test датасета в датафреймы

На выходе должны быть объявлены две переменные:
* train_dataset: pd.Dataframe
* test_dataset: pd.Dataframe

### Объявление путей

In [7]:
src_data_dir_path = os.path.join(PROJECT_DIR, 'data')
dist_data_dir = os.path.join(WORK_DIR, 'data')

data_archive_path = os.path.join(src_data_dir_path, 'E.zip')

### Разархивация

In [8]:
!unzip -o "$data_archive_path" -d "$dist_data_dir" > /dev/null

### Считывание

In [9]:
train_file_path = os.path.join(dist_data_dir, 'E', 'exposition_train.tsv')
test_file_path = os.path.join(dist_data_dir, 'E', 'exposition_test.tsv')

train_dataset = pd.read_csv(train_file_path, index_col='id', sep='\t')
test_dataset = pd.read_csv(test_file_path, index_col='id', sep='\t')

## Визуализация

### Просмотр первых строк

In [10]:
train_dataset.head()

,building_series_id,site_id,target,parking,target_string,build_year,expect_demolition,main_image,latitude,total_area,ceiling_height,rooms,floors_total,living_area,floor,is_apartment,building_id,has_elevator,studio,unified_address,area,kitchen_area,day,longitude,price,flats_count,building_type,balcony,locality_name,renovation
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5677548107212057955,1564812,0,1,OPEN,LESS_7,2005,False,//avatars.mds.yandex.net/get-realty/903734/add...,55.645313,105.000000,3.00,3,20,50.0,14,False,7969879732878112812,True,False,"Россия, Москва, Пролетарский проспект, 7",105.000000,15.0,2018-07-15,37.657490,95000,407,MONOLIT,BALCONY,Москва,EURO
155646401125694364,1564812,0,2,CLOSED,7_14,2010,False,//avatars.mds.yandex.net/get-realty/1702013/ad...,55.537102,40.000000,3.00,1,3,0.0,1,False,7667415960903930340,False,False,"Россия, Москва, посёлок Первомайское, Централь...",40.000000,10.0,2019-01-18,37.155632,25000,40,MONOLIT,UNKNOWN,посёлок Первомайское,COSMETIC_DONE
9186198458182518100,663302,0,2,OPEN,7_14,1995,False,//avatars.mds.yandex.net/get-realty/924080/add...,55.662956,37.599998,2.64,0,17,0.0,4,False,7166215405310646476,True,True,"Россия, Москва, улица Намёткина, 13к1",37.599998,0.0,2018-04-24,37.555466,26000,472,PANEL,LOGGIA,Москва,GOOD
10844743366553352344,1564812,0,2,OPEN,7_14,2018,False,//avatars.mds.yandex.net/get-realty/1521999/ad...,55.669151,80.000000,0.00,3,27,49.0,23,False,2039402855860137453,True,False,"Россия, Московская область, Одинцово, Верхне-П...",80.000000,20.0,2019-02-19,37.285000,35000,156,PANEL,UNKNOWN,Одинцово,GOOD
3712912186792420056,1564812,0,3,UNKNOWN,14_30,2004,False,//avatars.mds.yandex.net/get-realty/50286/f5c8...,55.828518,100.000000,3.00,3,4,0.0,3,False,4638454967482853510,True,False,"Россия, Москва, улица Рословка, 12к1",100.000000,0.0,2017-08-08,37.361897,80000,31,MONOLIT,UNKNOWN,Москва,EURO


### Проверка на пропуски

In [11]:
train_dataset.isna().sum()

building_series_id       0
site_id                  0
target                   0
parking                  0
target_string            0
build_year               0
expect_demolition        0
main_image            1097
latitude                 0
total_area               0
ceiling_height           0
rooms                    0
floors_total             0
living_area              0
floor                    0
is_apartment             0
building_id              0
has_elevator             0
studio                   0
unified_address          0
area                     0
kitchen_area             0
day                      0
longitude                0
price                    0
flats_count              0
building_type            0
balcony                  0
locality_name            0
renovation               0
dtype: int64

### Изучение различных полей

* building_series_id

In [12]:
assert (train_dataset.building_series_id == 0).sum() > 0
assert (test_dataset.building_series_id == 0).sum() > 0

print('Есть объявления, для которых не указан id серии дома')

Есть объявления, для которых не указан id серии дома


* site_id

In [13]:
assert (train_dataset.site_id == 0).sum() > 0
assert (test_dataset.site_id == 0).sum() > 0

print('Есть объявления, для которых не указан id жилого комплекса')

Есть объявления, для которых не указан id жилого комплекса


* build_year

In [14]:
assert (train_dataset.build_year == 0).sum() > 0
assert (test_dataset.build_year == 0).sum() > 0

print('Есть объявления, для которых не указан год постройки')

Есть объявления, для которых не указан год постройки


* parking

In [15]:
Counter(train_dataset.parking)

Counter({'CLOSED': 2170,
         'OPEN': 32650,
         'UNDERGROUND': 3346,
         'UNKNOWN': 318334})

* expect_demolition 

In [16]:
Counter(train_dataset.expect_demolition)

Counter({False: 341610, True: 14890})

* flats_count

In [17]:
assert (train_dataset.flats_count == 0).sum() > 0
assert (test_dataset.flats_count == 0).sum() > 0

print('Есть объявления, для которых не указано количество квартир в доме')

Есть объявления, для которых не указано количество квартир в доме


* floors_total

In [18]:
assert (train_dataset.floors_total == 0).sum() == 0
assert (test_dataset.floors_total == 0).sum() == 0

print('Для всех объявлений указано количество этажей в доме')

Для всех объявлений указано количество этажей в доме


* floor

In [19]:
assert (train_dataset.floor == 0).sum() == 0
assert (test_dataset.floor == 0).sum() == 0

print('Для всех объявлений указан этаж квартиры')

Для всех объявлений указан этаж квартиры


* total_area и area

In [20]:
assert len(train_dataset[train_dataset.total_area != train_dataset.area]) == 0
assert len(test_dataset[test_dataset.total_area != test_dataset.area]) == 0

print("Поля total_area и area дубликаты")

Поля total_area и area дубликаты


In [21]:
assert (train_dataset.total_area == 0).sum() == 0
assert (test_dataset.total_area == 0).sum() == 0

print("Для всех объявлений указана цена за квадратный метр")

Для всех объявлений указана цена за квадратный метр


* living_area


In [22]:
assert (train_dataset.living_area == 0).sum() > 0
assert (test_dataset.living_area == 0).sum() > 0

print("Есть объявления, для которых не указана жилая площадь")

Есть объявления, для которых не указана жилая площадь


* kitchen_area

In [23]:
assert (train_dataset.kitchen_area == 0).sum() > 0
assert (test_dataset.kitchen_area == 0).sum() > 0

print("Есть объявления, для которых не указана площадь кухни")

Есть объявления, для которых не указана площадь кухни


* ceiling_height

In [24]:
assert (train_dataset.ceiling_height == 0).sum() > 0
assert (test_dataset.ceiling_height == 0).sum() > 0

print('Есть объявления, для которых не указана высота потолка')

Есть объявления, для которых не указана высота потолка


In [25]:
print('Средняя высота потолков в трейне:', train_dataset[train_dataset.ceiling_height != 0].ceiling_height.mean())
print('Средняя высота потолков в тесте:', test_dataset[test_dataset.ceiling_height != 0].ceiling_height.mean())

Средняя высота потолков в трейне: 2.7693942716492055
Средняя высота потолков в тесте: 2.780900751373181


* price

In [26]:
assert (train_dataset.price == 0).sum() == 0
assert (test_dataset.price == 0).sum() == 0

print("Для всех объявлений указана цена")

Для всех объявлений указана цена


* day

In [27]:
print(train_dataset['day'].min())
print(train_dataset['day'].max())

2017-01-01
2019-10-31


In [28]:
print(test_dataset['day'].min())
print(test_dataset['day'].max())

2019-11-01
2020-03-31


* main_image

In [29]:
assert train_dataset['main_image'].isna().sum() > 0
print('В тренировочном датасете есть объявления без изображения')
assert test_dataset['main_image'].isna().sum() > 0
print('В тестовом датасете есть объявления без изображения')

В тренировочном датасете есть объявления без изображения
В тестовом датасете есть объявления без изображения


## Финальная обработка


На выходе должны быть объявлены переменные:

* **X_train_pure**: pd.DataFrame

* **y_train**: pd.Series

* **X_valid_pure**: pd.DataFrame

* **y_valid**: pd.Series

* **X_test_pure**: pd.DataFrame


### Извлечение выборки и таргета


In [30]:
train_dataset = train_dataset.sort_values(by='day')

partition = int(len(train_dataset) * (1 - VALIDATE_RATIO))
train_train_dataset = train_dataset.iloc[:partition]
train_valid_dataset = train_dataset.iloc[partition:]

X_train_train_pure = train_train_dataset.drop('target', axis=1)
X_train_train_pure = X_train_train_pure.drop('target_string', axis=1)
y_train_train = train_train_dataset['target']

X_train_valid_pure = train_valid_dataset.drop('target', axis=1)
X_train_valid_pure = X_train_valid_pure.drop('target_string', axis=1)
y_train_valid = train_valid_dataset['target']
X_test_pure = test_dataset

### Добавление дополнительных признаков и удаление ненужных

In [31]:
def add_time_features(X):
    datetime_series = pd.to_datetime(X['day'], format='%Y-%m-%d')
    X['Month'] = pd.DatetimeIndex(datetime_series).month
    X['Day'] = pd.DatetimeIndex(datetime_series).day
    X['DayOfWeek'] = pd.DatetimeIndex(datetime_series).dayofweek
    X['Season'] = (X['Month'] % 12 + 3) // 3
    return X

def add_adress_features(X):
    X['District'] = train_dataset.unified_address.map(lambda x: x.split(', ')[-2])
    return X

def add_price_per_area_features(X):
    X['PricePerTotalArea'] = X['price'] / X['area']
    return X

def add_floor_features(X):
    X['FloorPercantage'] = X['floor'] / X['floors_total']
    ceiling_height = train_dataset.ceiling_height.where(train_dataset.ceiling_height != 0, 2.77)
    X['FloorHeight'] = X['floor'] * ceiling_height
    return X

def add_missing_features(X):
    X['MainImageNa'] = X.main_image.isna()
    return X

def drop_columns(X):
    dropped_columns = [
        'day', 'total_area', 'main_image', 'building_id'
    ]
    return X.drop(dropped_columns, axis=1)

def prepare_sample(X):
    X = X.copy()
    X = add_time_features(X)
    X = add_adress_features(X)
    X = add_price_per_area_features(X)
    X = add_floor_features(X)
    X = add_missing_features(X)
    X = drop_columns(X)
    return X

In [32]:
X_train_train = prepare_sample(X_train_train_pure)
X_train_valid = prepare_sample(X_train_valid_pure)
X_test = prepare_sample(X_test_pure)

In [33]:
X_train_valid.columns

Index(['building_series_id', 'site_id', 'parking', 'build_year',
       'expect_demolition', 'latitude', 'ceiling_height', 'rooms',
       'floors_total', 'living_area', 'floor', 'is_apartment', 'has_elevator',
       'studio', 'unified_address', 'area', 'kitchen_area', 'longitude',
       'price', 'flats_count', 'building_type', 'balcony', 'locality_name',
       'renovation', 'Month', 'Day', 'DayOfWeek', 'Season', 'District',
       'PricePerTotalArea', 'FloorPercantage', 'FloorHeight', 'MainImageNa'],
      dtype='object')

# Эксперименты с обучением

На выходе должны быть объявлены переменные

* **y_test_predicted**: pd.Series \
Id | Target
* **SUBMISSTION_FILE_NAME**: string\
Названия файла для сохранения

## Catboost

### Объявление loss функции

In [34]:
class LossFunction(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)
        
        result = []
        for index in range(len(targets)):
            der1 = targets[index] - approxes[index]
            sign = -1 if  approxes[index] < targets[index] else 1
            der2 = math.exp(abs(targets[index] - approxes[index]))
            der1 = der2 * sign

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result

In [35]:
class EvalMetric(LossFunction):
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error = -(math.exp(abs(approx[i] - target[i])) - 1)
            error_sum += error

        return error_sum, weight_sum
    
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

### Объявление пула и параметров

In [38]:
cat_features = [
    'building_series_id', 'site_id', 'parking', 'expect_demolition',
    'is_apartment', 'unified_address', 'has_elevator', 'studio', 'building_type',
    'balcony', 'locality_name', 'renovation', 'District'
]
ignored_features = [
]
train_train_pool = Pool(
    X_train_train,
    y_train_train,
    cat_features=cat_features
)
train_valid_pool = Pool(
    X_train_valid,
    y_train_valid,
    cat_features=cat_features
)
params = {
    'iterations': 150,
    'learning_rate': 0.03,
    'eval_metric': EvalMetric(),
    'loss_function': LossFunction(),
#     'random_seed': 113,
    # 'logging_level': 'Silent',
    'use_best_model': False,
    'ignored_features': ignored_features
}

### Обучение с валидацией

In [39]:
%%time
model = CatBoostRegressor(**params)
model.fit(train_train_pool, eval_set=train_valid_pool)

0:	learn: -43.4781770	test: -42.6693065	best: -42.6693065 (0)	total: 1.44s	remaining: 3m 34s
1:	learn: -42.1636386	test: -41.3786760	best: -41.3786760 (1)	total: 2.79s	remaining: 3m 26s
2:	learn: -40.8879504	test: -40.1261776	best: -40.1261776 (2)	total: 4.17s	remaining: 3m 24s
3:	learn: -39.6499606	test: -38.9106834	best: -38.9106834 (3)	total: 5.57s	remaining: 3m 23s
4:	learn: -38.4485644	test: -37.7311338	best: -37.7311338 (4)	total: 6.93s	remaining: 3m 20s


CatBoostError: ignored

In [ ]:
model.get_feature_importance(train_valid_pool, prettified=True)

### Визуализация результатов

In [ ]:
def visualize_learning_results(metrics):
    """
    Plot one graph for each metric

    :param metrics: dic of metrics
    """

    n = len(metrics)
    
    fig = make_subplots(
        rows=n,
        cols=1,
        subplot_titles=list(metrics.keys())
    )

    for i, (metric_name, curves) in enumerate(metrics.items()):
        for dataset_type, curve in curves.items():
            m = len(curve)
            fig.add_trace(
                go.Scatter(
                    x=np.arange(m),
                    y=curve,
                    mode='lines',
                    name=f'{dataset_type} {metric_name}'
                ),
                row=i + 1,
                col=1
            )

    fig.update_layout(
        title_text="Learning results",
        width=297. * 3,
        height=210. * 3 * n
    )
    fig.show()

In [ ]:
eval_results = model.get_evals_result()

In [ ]:
eval_results = model.get_evals_result()
metrics = dict(
    Loss=dict(
        validation=eval_results['validation']['EvalMetric'],
        learn=eval_results['learn']['EvalMetric']
    ),
    Metric=dict(
        validation=eval_results['validation']['EvalMetric']
    )
)

KeyError: ignored

In [ ]:
visualize_learning_results(metrics)

NameError: ignored

### Объявление финальных параметров

In [36]:
cat_features = [
    'building_series_id', 'site_id', 'parking', 'expect_demolition',
    'is_apartment', 'unified_address', 'has_elevator', 'studio', 'building_type',
    'balcony', 'locality_name', 'renovation', 'District'
]
ignored_features = [
]
train_pool = Pool(
    pd.concat([X_train_train, X_train_valid]),
    pd.concat([y_train_train, y_train_valid]),
    cat_features=cat_features
)
params = {
    'iterations': 200,
    'learning_rate': 0.03,
    'eval_metric': EvalMetric(),
    'loss_function': LossFunction(),
    'random_seed': 113,
    # 'logging_level': 'Silent',
    'use_best_model': False,
    'ignored_features': ignored_features
}
n_models = 10

### Обучение

In [ ]:
%%time
models = []
np.random.seed(1)

for i in range(n_models):
    params['random_seed'] = np.random.randint(100000)
    model = CatBoostRegressor(**params)
    model.fit(train_pool)
    models.append(model)
    print("{}/{} Finished".format(i + 1, n_models))

0:	learn: -43.3164071	total: 2.42s	remaining: 8m 1s
1:	learn: -42.0066505	total: 4.69s	remaining: 7m 43s
2:	learn: -40.7356094	total: 7.09s	remaining: 7m 45s
3:	learn: -39.5021289	total: 9.24s	remaining: 7m 32s
4:	learn: -38.3051009	total: 11.5s	remaining: 7m 27s
5:	learn: -37.1434557	total: 13.4s	remaining: 7m 12s
6:	learn: -36.0161345	total: 15.9s	remaining: 7m 19s
7:	learn: -34.9221342	total: 18.2s	remaining: 7m 16s
8:	learn: -33.8604720	total: 20.4s	remaining: 7m 12s
9:	learn: -32.8301846	total: 22.3s	remaining: 7m 4s
10:	learn: -31.8303441	total: 24.3s	remaining: 6m 58s


### Получение предсказаний

In [ ]:
y_test_predicted = None
X_test_test = X_test.drop('public', axis=1)
for model in models:
    predicted = model.predict(X_test_test)
    y_test_cur_predicted = pd.Series(
        predicted,
        index=X_test_test.index,
        name='target'
    ).sort_index()
    
    if y_test_predicted is None:
        y_test_predicted = y_test_cur_predicted
    else:
        y_test_predicted += y_test_cur_predicted

y_test_predicted /= n_models
y_test_predicted = y_test_predicted.round().astype('int64')
SUBMISSTION_FILE_NAME = 'exposition_sample_submission.tsv'

# Отправка результатов

## Определение пути

In [ ]:
submission_folder_path = os.path.join(PROJECT_DIR, 'submissions')
file_path = os.path.join(submission_folder_path, SUBMISSTION_FILE_NAME)
print(f"File will be saved to {file_path}")

File will be saved to /content/drive/My Drive/projects/hack_the_realty_exposition/submissions/exposition_sample_submission.tsv


## Сохранение

In [ ]:
y_test_predicted.to_csv(file_path, sep='\t')

# Юнит тестирование


## Подключение библиотек

In [ ]:
import unittest

## Объявление тестирующего класса

In [ ]:
class TestNotebook(unittest.TestCase):
    def test_add(self):
        self.assertEqual(2 + 2, 4)

## Запуск тестов

In [ ]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_add (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
